In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Your task is to answer one or more of the following questions listed below: 

- What formula would you use for a metric to rank the safety of each neighborhood for a student, a family with a young child below the age of 15, and an elderly person? What would be the ranking result when this formula is used for each category of people? 
- To what extent and accuracy can we predict the level of change in crime for a neighborhood?
- What unique statistics and patterns were you able to identify in Seattle’s crime? 


In [17]:
#crime_data1 = pd.read_csv('Crime_Data.csv', parse_dates=[])
crime_data_spd = pd.read_csv('SPD_Crime_Data__2008-Present.csv', parse_dates=['Offense Start DateTime','Offense End DateTime'])
call_data = pd.read_csv('Call_Data.csv',parse_dates=['Original Time Queued','Arrived Time'])

In [4]:
crime_data1.shape, crime_data_spd.shape, call_data.shape

((523591, 11), (821377, 17), (4206691, 11))

Lets look and clean all old data (and some new too if dont have)

In [18]:
call_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4206691 entries, 0 to 4206690
Data columns (total 11 columns):
CAD Event Number               int64
Event Clearance Description    object
Call Type                      object
Priority                       int64
Initial Call Type              object
Final Call Type                object
Original Time Queued           datetime64[ns]
Arrived Time                   object
Precinct                       object
Sector                         object
Beat                           object
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 353.0+ MB


In [19]:
crime_data_spd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 821377 entries, 0 to 821376
Data columns (total 17 columns):
Report Number             821377 non-null object
Offense ID                821377 non-null int64
Offense Start DateTime    820978 non-null datetime64[ns]
Offense End DateTime      821377 non-null object
Report DateTime           821377 non-null object
Group A B                 821377 non-null object
Crime Against Category    821377 non-null object
Offense Parent Group      821377 non-null object
Offense                   821377 non-null object
Offense Code              821377 non-null object
Precinct                  821377 non-null object
Sector                    821377 non-null object
Beat                      821377 non-null object
MCPP                      821377 non-null object
100 Block Address         792736 non-null object
Longitude                 821377 non-null float64
Latitude                  821377 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1)

lets take 2015 - 2019 Data

In [21]:
(crime_data_spd['Offense Start DateTime']> '2015-01-01' ).sum()

374716

In [22]:
(call_data['Original Time Queued']>'2015-01-01').sum()

2085932

lets work with `crime_data_spd` and `call_data`

In [23]:
### drop nan in date, filter for 2015-2019 , reindex with date
crime_data_spd.set_index(crime_data_spd['Offense Start DateTime'],inplace=True)


In [24]:
call_data.set_index(call_data['Original Time Queued'],inplace=True)

In [27]:
call_data.head()

,CAD Event Number,Event Clearance Description,Call Type,Priority,Initial Call Type,Final Call Type,Original Time Queued,Arrived Time,Precinct,Sector,Beat
Original Time Queued,,,,,,,,,,,
2009-06-02 03:35:46,2009000189335,Evidence Submitted (new incident number required),"TELEPHONE OTHER, NOT 911",1,DIST - IP/JO - DV DIST - NO ASLT,"--DV - ARGUMENTS, DISTURBANCE (NO ARREST)",2009-06-02 03:35:46,Jun 2 2009 04:09:38:000AM,EAST,CHARLIE,C1
2009-06-02 03:43:08,2009000189336,CITATION ISSUED (CRIMINAL OR NON-CRIMINAL),"TELEPHONE OTHER, NOT 911",3,TRAFFIC - BLOCKING ROADWAY,TRAFFIC - BLOCKING ROADWAY,2009-06-02 03:43:08,Jun 2 2009 05:47:21:000AM,WEST,QUEEN,Q1
2009-06-02 03:47:24,2009000189340,PROBLEM SOLVING PROJECT,ONVIEW,7,"PREMISE CHECK, OFFICER INITIATED ONVIEW ONLY","PREMISE CHECK, OFFICER INITIATED ONVIEW ONLY",2009-06-02 03:47:24,Jun 2 2009 03:47:24:000AM,WEST,KING,K1
2009-06-02 03:48:39,2009000189341,ASSISTANCE RENDERED,"TELEPHONE OTHER, NOT 911",1,BURG - IP/JO - RES (INCL UNOCC STRUCTURES),--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,2009-06-02 03:48:39,Jun 2 2009 04:01:55:000AM,SOUTH,OCEAN,O3
2009-06-02 03:49:37,2009000189342,ASSISTANCE RENDERED,"TELEPHONE OTHER, NOT 911",1,BURG - IP/JO - RES (INCL UNOCC STRUCTURES),--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,2009-06-02 03:49:37,Jun 2 2009 04:22:28:000AM,WEST,DAVID,D1


In [30]:
crime_data_spd.sort_index(inplace=True)

In [33]:
call_data.sort_index(inplace=True)

In [36]:
crime_1519 = crime_data_spd['2015-01-01':'2019-12-31']
call_1519 = call_data['2015-01-01':'2019-12-31']

In [37]:
crime_1519.to_csv('crime1519.csv')
call_1519.to_csv('call1519.csv')